In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
df = pd.DataFrame(columns=["document", "full_text", "tokens", "trailing_whitespace", "labels"])

In [3]:
for filename in tqdm(os.listdir("./vincent")):
    full_path = os.path.join("./vincent", filename)
    if full_path.endswith(".json"):
        new_df = pd.read_json(full_path)
        if "trailing whitespace" in new_df.columns:
            new_df = new_df.drop(columns=["trailing whitespace"])
        df = pd.concat([df, new_df], ignore_index=True)


def check_labels(label_list):
    return "I-EMAIL" not in label_list and "I-USERNAME" not in label_list


def has_no_adjacent_names(label_list):
    last_label = None
    for label in label_list:
        if label.startswith("B-NAME_STUDENT"):
            if last_label is not None and last_label.startswith("B-NAME_INSTRUCTOR"):
                return False
        if label.startswith("B-NAME_INSTRUCTOR"):
            if last_label is not None and last_label.startswith("B-NAME_STUDENT"):
                return False
        last_label = label
    return True

df.drop_duplicates(subset=["full_text"], inplace=True)
df["document"] = range(len(df))
df["document"] = df["document"].astype("str")
# df = df[df["labels"].apply(check_labels)]
df = df[df["labels"].apply(has_no_adjacent_names)]
df.reset_index(drop=True, inplace=True)
df.shape

100%|██████████| 4/4 [00:01<00:00,  2.67it/s]


(5528, 5)

In [4]:
df.head(10)

,document,full_text,tokens,trailing_whitespace,labels
0,0,"In the realm of problem-solving, one often enc...","[In, the, realm, of, problem, -, solving, ,, o...","[True, True, True, True, False, False, False, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,1,"In today's fast-paced world, where challenges ...","[In, today, 's, fast, -, paced, world, ,, wher...","[True, False, True, False, False, True, False,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,2,"As a student pursuing higher education, I am c...","[As, a, student, pursuing, higher, education, ...","[True, True, True, True, True, False, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,3,"As a student pursuing higher education, I am c...","[As, a, student, pursuing, higher, education, ...","[True, True, True, True, True, False, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,4,"In tackling complex challenges, I have always ...","[In, tackling, complex, challenges, ,, I, have...","[True, True, True, False, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5,5,"As a student, I have always been intrigued by ...","[As, a, student, ,, I, have, always, been, int...","[True, True, False, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6,6,"In today's fast-paced and ever-evolving world,...","[In, today, 's, fast, -, paced, and, ever, -, ...","[True, False, True, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
7,7,"In the realm of problem-solving, one often enc...","[In, the, realm, of, problem, -, solving, ,, o...","[True, True, True, True, False, False, False, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
8,8,As I sat down to tackle the complex challenge ...,"[As, I, sat, down, to, tackle, the, complex, c...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9,9,"Throughout my academic journey, I have encount...","[Throughout, my, academic, journey, ,, I, have...","[True, True, True, False, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [5]:
df.dtypes

document               object
full_text              object
tokens                 object
trailing_whitespace    object
labels                 object
dtype: object

In [6]:
df.to_json("./vincent/vincent_5k_0223.json", orient="records")